In [ ]:
import tkinter as tk
import pygame
pygame.mixer.init()
from tkinter import messagebox
from PIL import Image, ImageTk, ImageFilter
import copy
import math
import random
import heapq
import os

# ===== Constants =====
BOARD_SIZE = 15
CELL = 50
CANVAS_W = BOARD_SIZE*CELL
CANVAS_H = BOARD_SIZE*CELL

MAX_DEPTH = 3
ANIM_STEPS = 6
ANIM_DELAY = 30
AI_DELAY = 200

NUM_CRYSTALS = 20
NUM_OBSTACLES = 30

HUMAN_IMG_FILE = "human.png"
ROBOT_IMG_FILE = "robot.png"
ROBOT2_IMG_FILE = "robot2.png"       
CRYSTAL_IMG_FILE = "crystal.png"
OBSTACLE_IMG_FILE = "obstacle.png"
START_IMG_FILE = "Start.png"

START_SOUND= pygame.mixer.Sound("click.wav")
CRYSTAL_SOUND = pygame.mixer.Sound("crystal.wav")
WIN_SOUND = pygame.mixer.Sound("win.wav")
FAIR_SOUND = pygame.mixer.Sound("Fair.wav")

# ===== Main Game Class =====
class CrystalCapture:
    def __init__(self, root):
        self.root = root
        self.root.title("🌟Crystal Capture 🌟")
        
        # game state
        self.state = {}
        self.ai_running = False 
        
        # assets
        self.ASSETS = {"human": None, "robot": None, "crystal": None, "obstacle": None, "robot2": None}
        self.CANVAS_IDS = {"player_H": None, "player_A": None, "crystals": {}, "obstacles": {}}  
    
# ===== A* Pathfinding =====
    def manhattan(self,a,b):
        return abs(a[0]-b[0])+abs(a[1]-b[1])
    def Astar(self,start,goals,obstacles):
        open_set = [(0,start)]
        came_from = {}
        g_score = {start:0}
        f_score = {start:min(self.manhattan(start,g) for g in goals) if goals else 0}
        while open_set:
            _, current = heapq.heappop(open_set)
            if current in goals:
                path=[]
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.reverse()
                return path
            for neighbor in self.get_moves(current,obstacles,(-1,-1)):
                tentative_g = g_score[current]+1
                if neighbor not in g_score or tentative_g<g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + (min(self.manhattan(neighbor,g) for g in goals) if goals else 0)
                    heapq.heappush(open_set,(f_score[neighbor],neighbor))
        return []



    def minimax_ab(self, state, depth, alpha, beta, maximizing):
        if depth==0 or self.is_terminal():
            return self.evaluate(), None
        player = "B" if maximizing else "R"
        pos = state["B_pos"] if maximizing else state["R_pos"]
        other = state["R_pos"] if maximizing else state["B_pos"]
        if state["crystals"]:
            path = self.Astar(pos,state["crystals"],state["obstacles"])
            if path: return self.evaluate(), path[0]
        moves = self.get_moves(pos,state["obstacles"],other)
        if not moves: return self.evaluate(), None
        best_move = None
        if maximizing:
            max_val = -math.inf
            for mv in moves:
                new_state = self.simulate_move(mv,player)
                val,_ = self.minimax_ab(new_state,depth-1,alpha,beta,False)
                if val>max_val:
                    max_val = val
                    best_move = mv
                alpha = max(alpha,val)
                if beta<=alpha: break
            return max_val,best_move
        else:
            min_val = math.inf
            for mv in moves:
                new_state = self.simulate_move(mv,player)
                val,_ = self.minimax_ab(new_state,depth-1,alpha,beta,True)
                if val<min_val:
                    min_val=val
                    best_move=mv
                beta = min(beta,val)
                if beta<=alpha: break
            return min_val,best_move
        



    def ai_turn_step(self):
        if self.is_terminal() or self.ai_running or not self.game_active: return
        current = self.state["player"]
        if self.mode_var.get()=="Human vs AI" and current=="R": return
        _,mv=self.minimax_ab(copy.deepcopy(self.state),MAX_DEPTH,-1e9,1e9,current=="B")
        if mv:
            start_pos = self.state["R_pos"] if current == "R" else self.state["B_pos"]
            other_pos = self.state["B_pos"] if current == "R" else self.state["R_pos"]
        if mv == other_pos: 
            possible_moves = self.get_moves(start_pos, self.state["obstacles"], other_pos) 
            if possible_moves:
                mv = random.choice(possible_moves)
            else:
                mv=None
        if mv:
            self.animate_move(current, start_pos, mv)
        self.root.after(ANIM_STEPS*ANIM_DELAY + AI_DELAY, self.ai_turn_step) 


           

C:\Python312\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: No file 'click.wav' found in working directory 'C:\Users\mahmo\anaconda_projects\project_AI'.